In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
# We're going to be calculating memory usage a lot,
# so we'll create a function to save us some time!

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [3]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
postulaciones = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulaciones/Postulaciones-merge.csv',
                            dtype=column_types,parse_dates=['fechapostulacion'],infer_datetime_format=True)
postulaciones = postulaciones.drop(columns=['Unnamed: 0','fechapostulacion'])

In [4]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 2 columns):
idaviso         uint32
idpostulante    object
dtypes: object(1), uint32(1)
memory usage: 75.6+ MB


In [5]:
dtypes = pd.Series({'idpostulante': np.dtype('object'), 'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_genero_edad = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulantes-Genero-Edad/postulantes_genero_y_edad-merge.csv',
                                    dtype=column_types,parse_dates=['fechanacimiento'],
                                      infer_datetime_format=True)
postulantes_genero_edad = postulantes_genero_edad.drop(columns=['Unnamed: 0'])

In [6]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario',
                  'Terciario/Técnico', 'Universitario'],
                 ordered=False)})


dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_educacion = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulantes-Educacion/postulantes_educacion-merge.csv',
                                    dtype=column_types)
postulantes_educacion = postulantes_educacion.drop(columns=['Unnamed: 0','nombre_numerico'])

In [7]:
#Hago un merge de educacion y genero-edad
postulantes_merge = pd.merge(postulantes_genero_edad,postulantes_educacion,how='left',on='idpostulante')

In [8]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
fechanacimiento    504407 non-null object
idpostulante       504407 non-null object
sexo               504407 non-null category
estado             447909 non-null category
nombre             447909 non-null category
dtypes: category(3), object(2)
memory usage: 13.0+ MB


In [9]:
postulantes_merge.head()

,fechanacimiento,idpostulante,sexo,estado,nombre
0,1970-12-03,NM5M,FEM,Graduado,Secundario
1,1962-12-04,5awk,FEM,Graduado,Universitario
2,1978-08-10,ZaO5,FEM,Abandonado,Universitario
3,1969-05-09,NdJl,MASC,En Curso,Posgrado
4,1981-02-16,eo2p,MASC,Graduado,Master


In [10]:
#Voy a agregar una columna con la edad ya calculada(estimando por 2017-año nacimiento) para cada registro
postulantes_merge["año"] = postulantes_merge[postulantes_merge["fechanacimiento"] != 'nan']\
        ["fechanacimiento"].apply(lambda x: x[0:4])
postulantes_merge["edad"] = postulantes_merge[postulantes_merge["año"].notna()] \
        ["año"].apply(lambda x: 2017 - int(x))

In [11]:
#Elimino columna fechanacimiento y año que ya no me sirven
postulantes_merge = postulantes_merge.drop(columns=['fechanacimiento','año'])

In [12]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 5 columns):
idpostulante    504407 non-null object
sexo            504407 non-null category
estado          447909 non-null category
nombre          447909 non-null category
edad            477724 non-null float64
dtypes: category(3), float64(1), object(1)
memory usage: 13.0+ MB


# Agrego la informacion a postulaciones

In [13]:
merge_postulaciones = pd.merge(postulaciones,postulantes_merge,how='left',on='idpostulante')

In [14]:
#Creo columna con un 1 para indicar postulacion
merge_postulaciones['se_postulo'] = 1
merge_postulaciones['se_postulo'] = merge_postulaciones[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')

In [15]:
merge_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 7 columns):
idaviso         uint32
idpostulante    object
sexo            category
estado          category
nombre          category
edad            float64
se_postulo      uint8
dtypes: category(3), float64(1), object(1), uint32(1), uint8(1)
memory usage: 201.5+ MB


In [16]:
merge_postulaciones[merge_postulaciones['edad'].isnull()]

,idaviso,idpostulante,sexo,estado,nombre,edad,se_postulo
700,1112281337,NM1J0z,MASC,Graduado,Secundario,NaN,1
701,1112334640,NM1J0z,MASC,Graduado,Secundario,NaN,1
702,1112289378,NM1J0z,MASC,Graduado,Secundario,NaN,1
703,1112296073,NM1J0z,MASC,Graduado,Secundario,NaN,1
704,1112310231,NM1J0z,MASC,Graduado,Secundario,NaN,1
705,1112195462,NM1J0z,MASC,Graduado,Secundario,NaN,1
706,1112288872,NM1J0z,MASC,Graduado,Secundario,NaN,1
707,1112277388,NM1J0z,MASC,Graduado,Secundario,NaN,1
708,1112334643,NM1J0z,MASC,Graduado,Secundario,NaN,1
709,1112153209,NM1J0z,MASC,Graduado,Secundario,NaN,1


In [17]:
#Me quedo con los registros con todos los datos (por ahora desp ver si se puede usar imputing)
merge_sin_nan = merge_postulaciones[merge_postulaciones['nombre'].notnull() &
                                    merge_postulaciones['estado'].notnull() & 
                                    merge_postulaciones['edad'].notnull()]
merge_sin_nan = merge_sin_nan[merge_sin_nan['nombre'].notnull() &
                                    merge_sin_nan['estado'].notnull()]
merge_sin_nan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6322780 entries, 0 to 6603749
Data columns (total 7 columns):
idaviso         uint32
idpostulante    object
sexo            category
estado          category
nombre          category
edad            float64
se_postulo      uint8
dtypes: category(3), float64(1), object(1), uint32(1), uint8(1)
memory usage: 193.0+ MB


In [18]:
#Vamos a codificar cada columna que sea de strings
lb_make1 = LabelEncoder()
merge_sin_nan["estado_code"] = lb_make1.fit_transform(merge_sin_nan["estado"])

lb_make2 = LabelEncoder()
merge_sin_nan["sexo_code"] = lb_make2.fit_transform(merge_sin_nan["sexo"])

lb_make3 = LabelEncoder()
merge_sin_nan["nombre_code"] = lb_make3.fit_transform(merge_sin_nan["nombre"])

merge_sin_nan.drop(columns=['sexo','nombre','estado'],inplace=True)
merge_sin_nan[['sexo_code','nombre_code','estado_code','edad']] = merge_sin_nan[['sexo_code','nombre_code','estado_code','edad']].apply(pd.to_numeric,downcast='unsigned')

# Agrego datos de vistas que serian los postulantes que ven algo y no se postulan

In [19]:
dtypes = pd.Series({'idAviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

vistas = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Vistas/Vistas-merge.csv',dtype=column_types,parse_dates=['timestamp'],
                     infer_datetime_format=True)
vistas = vistas.drop(columns=['Unnamed: 0'])
#Considero unica vista de aviso por postulante (no me importa ni horario ni cantidad solo que lo vea)
vistas.drop(columns=['timestamp'],inplace=True)
vistas.drop_duplicates(subset=['idAviso','idpostulante'],inplace=True)
vistas['se_postulo'] = 0
vistas['se_postulo'] = vistas[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')

In [20]:
vistas.rename(index=str,columns={'idAviso':'idaviso'},inplace=True)
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         uint32
idpostulante    object
se_postulo      uint8
dtypes: object(1), uint32(1), uint8(1)
memory usage: 172.2+ MB


# Agrego datos de avisos y lo mergeo con las vistas

In [21]:
avisos = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/optimized_avisos_detalle.csv')

In [22]:
avisos.isnull().sum()

Unnamed: 0                   0
idaviso                      0
nombre_area_code             0
denominacion_empresa_code    0
nivel_laboral_code           0
tipo_de_trabajo_code         0
nombre_zona_code             0
dtype: int64

In [23]:
#Hago un merge de avisos y vistas
vistas_y_avisos = pd.merge(vistas,avisos,how='inner',on='idaviso')

In [24]:
vistas_y_avisos.head(5)

,idaviso,idpostulante,se_postulo,Unnamed: 0,nombre_area_code,denominacion_empresa_code,nivel_laboral_code,tipo_de_trabajo_code,nombre_zona_code
0,1112400775,kPjr2K0,0,16018,185,925,4,1,7
1,1112400775,EzEovD2,0,16018,185,925,4,1,7
2,1112400775,wVkA0vQ,0,16018,185,925,4,1,7
3,1112400775,EzeBkq8,0,16018,185,925,4,1,7
4,1112400775,NzromrM,0,16018,185,925,4,1,7


In [25]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         uint32
idpostulante    object
se_postulo      uint8
dtypes: object(1), uint32(1), uint8(1)
memory usage: 172.2+ MB


# Merge de todos

In [26]:
#Puede que haya vistas que enrealidad el postulante luego se postulo asi que verifico con el dataset merge
outer = pd.merge(merge_sin_nan,vistas_y_avisos,how='outer',on=['idaviso','idpostulante'])

In [27]:
#Me quedo con los registros con todos los datos (por ahora desp ver si se puede usar imputing)
outer = outer[outer['nombre_code'].notnull() &
            outer['estado_code'].notnull() & 
            outer['edad'].notnull() &
             outer['nombre_area_code'].notnull() &
             outer['denominacion_empresa_code'].notnull() &
             outer['nivel_laboral_code'].notnull() &
             outer['tipo_de_trabajo_code'].notnull() &
             outer['nombre_zona_code'].notnull()]

In [30]:
outer.se_postulo_x.value_counts()

1.0    1674375
Name: se_postulo_x, dtype: int64

In [31]:
#Dejo solo los que no aparecen ya en postulaciones
trainingSet = outer[outer['se_postulo_x'].isnull() & outer['se_postulo_y'] == 0]

In [32]:
trainingSet.drop('se_postulo_x',axis=1,inplace=True)
trainingSet.drop('Unnamed: 0',axis=1,inplace=True)
trainingSet.rename(index=str,columns={'se_postulo_y':'se_postulo'},inplace=True)

In [33]:
trainingSet.se_postulo.value_counts()

0.0    1674375
Name: se_postulo, dtype: int64

In [34]:
postMasVistas = pd.concat([trainingSet,merge_sin_nan])

In [37]:
postMasVistas.isnull().sum()

denominacion_empresa_code    6322780
edad                               0
estado_code                        0
idaviso                            0
idpostulante                       0
nivel_laboral_code           6322780
nombre_area_code             6322780
nombre_code                        0
nombre_zona_code             6322780
se_postulo                         0
sexo_code                          0
tipo_de_trabajo_code         6322780
dtype: int64

In [38]:
postMasVistas = postMasVistas[postMasVistas['denominacion_empresa_code'].notnull() &
                             postMasVistas['nivel_laboral_code'].notnull() &
                             postMasVistas['nombre_area_code'].notnull() &
                             postMasVistas['nombre_zona_code'].notnull() &
                             postMasVistas['tipo_de_trabajo_code'].notnull()]

In [39]:
postMasVistas.isnull().sum()

denominacion_empresa_code    0
edad                         0
estado_code                  0
idaviso                      0
idpostulante                 0
nivel_laboral_code           0
nombre_area_code             0
nombre_code                  0
nombre_zona_code             0
se_postulo                   0
sexo_code                    0
tipo_de_trabajo_code         0
dtype: int64

In [40]:
postMasVistas.se_postulo.value_counts()

0.0    1674375
Name: se_postulo, dtype: int64

In [42]:
#postMasVistas.to_csv('/home/fabrizio/env/NaventDatosTP/Data/postulaciones_mas_vistas_mas_avisos.csv')

In [92]:
postMasVistasCopiaSinPostulanteParaCorr = postMasVistas.drop('idpostulante',axis=1)

In [95]:
postMasVistasCopiaSinPostulanteParaCorr = postMasVistas.drop('idaviso',axis=1)

In [97]:
postMasVistasCopiaSinPostulanteParaCorr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1674375 entries, 23 to 6322779
Data columns (total 10 columns):
denominacion_empresa_code    1674375 non-null float64
edad                         1674375 non-null float64
estado_code                  1674375 non-null float64
nivel_laboral_code           1674375 non-null float64
nombre_area_code             1674375 non-null float64
nombre_code                  1674375 non-null float64
nombre_zona_code             1674375 non-null float64
se_postulo                   1674375 non-null float64
sexo_code                    1674375 non-null float64
tipo_de_trabajo_code         1674375 non-null float64
dtypes: float64(10)
memory usage: 140.5+ MB


In [104]:
postMasVistasCopiaSinPostulanteParaCorr.se_postulo.value_counts()

0.0    1674375
Name: se_postulo, dtype: int64

In [ ]:
#Dropeo las columnas que use para codificar
postulantes_merge = postulantes_merge.drop(columns=['sexo','nombre','estado'])
postulantes_merge.info()

In [ ]:
postulantes_merge_int = postulantes_merge.select_dtypes(include=['int','float'])
converted_int = postulantes_merge_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(postulantes_merge_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([postulantes_merge_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
optimized_postulantes_merge = postulantes_merge.copy()

optimized_postulantes_merge[converted_int.columns] = converted_int

print(mem_usage(postulantes_merge))
print(mem_usage(optimized_postulantes_merge))

In [ ]:
optimized_postulantes_merge.info()

In [19]:
optimized_postulantes_merge.to_csv('Data/optimized_postulantes_merge.csv')

In [20]:
intereses_vistas = pd.read_csv('Data/unique_postulantes_con_intereses_vistas.csv')

In [22]:
intereses_vistas.head()

,idpostulante,nombre_zona_interes,nivel_laboral_interes,tipo_de_trabajo_interes,nombre_area_interes
0,YjVJQ6Z,-7,-4,-1,-175
1,BmVpYoR,NaN,NaN,NaN,NaN
2,wVkBzZd,-7,-2,-1,-120
3,OqmP9pv,-7,-4-2,-1,-175
4,DrpbXDP,NaN,NaN,NaN,NaN
